# Цифровая подпись Эль Гамаля

### Функции для поиска простых чисел и мульт. обратного

In [1]:
import random
from math import gcd


# Тест Люка — Лемера для чисел Мерсена
def lucas_lemer(prime):
    S = 4
    k = 1
    M = 2**prime-1
    while k != (prime-1):
        S = ((S*S)-2) % M
        k += 1
    return S == 0


# Поиск простого числа
def random_prime():
    while True:
        prime = random.randint(64, 512)
        if 2 in [prime, 2**prime % prime]:
            if lucas_lemer(prime):
                break
    return 2**prime-1

# Пара взаимно простых чисел a, b a > b
def gen_pair_prime():
    a = random_prime()
    while True:
        b = random_prime()
        if a != b and gcd(a, b) == 1:
            if a > b:
                return a, b
            else:
                return b, a


# Мультипликативное обратное
def modinv(a, m):
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    
    g, x, _ = egcd(a, m)
    if g != 1:
        raise Exception("Error")
    else:
        return x % m


# Генерация случайного k и мультипликативного обратного к нему по модулю p-1
def random_k_invk(p):
    k = random.randint(2, p-2)
    if gcd(k, p-1) != 1:
        return random_k_invk(p)
    else:
        return k, modinv(k, p-1)

#### Класс пользователя

In [2]:
class User():
    def __init__(self, g, p):
        self.g = g
        self.p = p
        self.__secret_key = random.randint(2, p - 1)
        self.public_key = pow(g, self.__secret_key, p)

    # Хеш функция
    def __hash(self, message):
        h = sum([ord(i) for i in message]) % (p - 2)
        if h == 1:
            h += 1
        return h
    
    # Генерация нового секретного и открытого ключа
    def gen_new_key(self):
        self.__secret_key = random.randint(2, p - 1)
        self.public_key = pow(g, self.__secret_key, p)

    # Создание подписи к сообщению
    def make_sing(self, message):
        k, k_inv = random_k_invk(self.p)
        r = pow(g, k, p)
        m = self.__hash(message)
        s = (m - self.__secret_key*r) * k_inv % (p - 1)
        # print(f"K: {k} iK: {k_inv} m: {m} r: {r} s: {s} sign({r},{s})\n")
        k = None
        return r, s
    
    # Проверка подписи
    def check_sign(self, publicKey, message, sign):
        r, s = sign
        m = self.__hash(message)
        # print("hash", m)
        # print("yrp", publicKey, r, self.p)
        # print("rsp", r, s, self.p)
        left = pow(publicKey, r, self.p)*pow(r, s, self.p) % self.p
        right = pow(self.g, m, self.p)
        # print("left:", left, "right:",  right)
        return left == right

Генерация простого g и первообразного корня к нему p

In [3]:
# g = random_prime()
# p = random.randint(4, 13)
# g, p = 618970019642690137449562111, 11

g, p = gen_pair_prime()
print(g, p)
print(gcd(g, p))

170141183460469231731687303715884105727 162259276829213363391578010288127
1


Создание пользователей

In [4]:
Alice = User(g, p)
Bob = User(g, p)
Carol = User(g, p)

#### Alice создает подпись к сообщению. Bob проверяет что это сообщение от Alice

In [5]:
mesg_A = "Hello World!!!"
sign_A = Alice.make_sing(mesg_A)

res_check = Bob.check_sign(Alice.public_key, mesg_A, sign_A)

if res_check:
    print("Это сообщение от Alice")
else:
    print("Это сообщение НЕ от Alice")

Это сообщение от Alice


#### Carol заменяет сообщение, но не трогает подпись. Bob проверяет что это сообщение от Alice

In [6]:
mesg_С = "Hello World!!?"

res_check = Bob.check_sign(Alice.public_key, mesg_С, sign_A)

if res_check:
    print("Это сообщение от Alice")
else:
    print("Это сообщение НЕ от Alice")

Это сообщение НЕ от Alice


#### Carol знает хеш-функцию. Carol заменяет сообщение, и делает свою подпись. Bob проверяет что это сообщение от Alice

In [7]:
mesg_С = "Hello World!!?"
sign_С = Carol.make_sing(mesg_С)

res_check = Bob.check_sign(Alice.public_key, mesg_С, sign_С)

if res_check:
    print("Это сообщение от Alice")
else:
    print("Это сообщение НЕ от Alice")

Это сообщение НЕ от Alice


#### Carol посылает сообщение Alice

In [8]:
mesg_С = "Hello World!!?"
sign_С = Carol.make_sing(mesg_С)

res_check = Alice.check_sign(Carol.public_key, mesg_С, sign_С)

if res_check:
    print("Это сообщение от Carol")
else:
    print("Это сообщение НЕ от Carol")

Это сообщение от Carol
